In [1]:
!git clone https://github.com/jameslyons/python_speech_features.git

Cloning into 'python_speech_features'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 427 (delta 0), reused 0 (delta 0), pack-reused 426
Receiving objects: 100% (427/427), 219.78 KiB | 4.78 MiB/s, done.
Resolving deltas: 100% (206/206), done.


In [2]:
!sudo apt-get install libsndfile-dev


Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libsndfile1-dev' instead of 'libsndfile-dev'
libsndfile1-dev is already the newest version (1.0.28-4).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [3]:
!git clone https://github.com/JoergFranke/phoneme_recognition.git && cd phoneme_recognition && pip install -r requirements.txt


Cloning into 'phoneme_recognition'...
remote: Enumerating objects: 74, done.
remote: Total 74 (delta 0), reused 0 (delta 0), pack-reused 74
Unpacking objects: 100% (74/74), done.
    100% |████████████████████████████████| 71kB 2.8MB/s 
    100% |████████████████████████████████| 112kB 7.3MB/s 
    100% |████████████████████████████████| 2.9MB 8.8MB/s 
    100% |████████████████████████████████| 901kB 19.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/b2/2b/20/e13dcf8a99bfa8f417242f1ea2894e9a82793cad447a5872f4
  Stored in directory: /root/.cache/pip/wheels/ed/30/07/f2bdd0b1cc48fd897dc36be0f5575f768be09b4931301abaa7
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
  Stored in directory: /root/.cache/pip/wheels/e4/33/9d/880c86cc0db0b29d34fb6c62256322a4ea1694404192c2b408
  Stored in directory: /root/.cache/pip/wheels/19/95/78/704c96f15db4fae2d17cdb1e8c48ec0b8ce02369f8d7139285
Successfully built klepto pox python-speech-featur

In [0]:
!unzip "./drive/My Drive/COLAB Neural/preprocessed_files_with_dict.zip" -d "./drive/My Drive/COLAB Neural/"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import shutil
import numpy as np
from scikits.audiolab import Sndfile
import python_speech_features as sf

In [12]:
dict_list = {'DR1': "1,1,1,1 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0", 
            'DR2': "0,0,0,0 1,1,1,1 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0", 
            'DR3': "0,0,0,0 0,0,0,0 1,1,1,1 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0", 
            'DR4': "0,0,0,0 0,0,0,0 0,0,0,0 1,1,1,1 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0", 
            'DR5': "0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 1,1,1,1 0,0,0,0 0,0,0,0 0,0,0,0", 
            'DR6': "0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 1,1,1,1 0,0,0,0 0,0,0,0", 
            'DR7': "0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 1,1,1,1 0,0,0,0", 
            'DR8': "0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 0,0,0,0 1,1,1,1" 
            }
dirlist = ['DR1', 'DR2', 'DR3', 'DR4', 'DR5', 'DR6', 'DR7', 'DR8']

# creazione dei quaternioni appesi alla lista final_quat
def make_quaternion(mat):
	#print(mat.shape)
	res = []
	for row in range(len(mat)):
		for j in range(41):
			quat = [0,mat[row][j],mat[row][j+41],mat[row][j+82]]
			res.append(quat)
	return res

if __name__ == '__main__':
    ####################################
    # pre process parameters    
	
    # Mel-Frequency Cepstrum Coefficients, default 12
    numcep=40 #40
	
    # the number of filters in the filterbank, default 26
    numfilt = 40 #40

	# the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)
    winlen = 0.025
	
    # the step between successive windows in seconds. Default is 0.01s (10 milliseconds)
    winstep = 0.01
	
    # use  first or first+second order derivation
    grad = 2

	#number of elements for each set of quaternions
    N_SPLIT = 400

    dir_name = "./drive/My Drive/COLAB Neural/Quaternion_" + str(N_SPLIT) + "_Lamush"
    tree = "./drive/My Drive/COLAB Neural/" + dir_name
#     tree = "/home/franci/Desktop/NN/Quaternion-speech-recognition/" + dir_name
    if os.path.isdir(tree):
        shutil.rmtree(tree)
    
    os.makedirs(dir_name)
    path_to_quats = os.path.abspath(dir_name)
    
    path_to_prep_files = os.path.abspath("./drive/My Drive/COLAB Neural/preprocessed_files_with_dict/")
    dir = os.listdir(path_to_prep_files)

    for path_dict in dir:
        print("Analizzo la cartella " + str(path_dict))
        dir_dict = os.listdir(path_to_prep_files + "/" + str(path_dict))
        for file in dir_dict:
            if file.split(".")[1] == "WAV":
                p = path_to_prep_files + "/" + str(path_dict) + "/" + file
                f = Sndfile(p, 'r')
                frames = f.nframes
                samplerate = f.samplerate
                data = f.read_frames(frames)
                data = np.asarray(data)
                #print(data.shape)
                #calc mfcc
                feat_raw,energy = sf.fbank(data, samplerate,winlen,winstep, nfilt=numfilt)
                feat = np.log(feat_raw)
                feat = sf.dct(feat, type=2, axis=1, norm='ortho')[:,:numcep]
                feat = sf.lifter(feat,L=22)
                feat = np.asarray(feat)
                #calc log energy
                log_energy = np.log(energy) #np.log( np.sum(feat_raw**2, axis=1) )
                log_energy = log_energy.reshape([log_energy.shape[0],1])
                mat = ( feat - np.mean(feat, axis=0) ) / (0.5 * np.std(feat, axis=0))
                mat = np.concatenate((mat, log_energy), axis=1)
                #calc first order derivatives
                if grad >= 1:
                    gradf = np.gradient(mat)[0]
                    mat = np.concatenate((mat, gradf), axis=1)
                #calc second order derivatives
                if grad == 2:
                    grad2f = np.gradient(gradf)[0]
                    mat = np.concatenate((mat, grad2f), axis=1)

                file_to_create = path_to_quats + "/" + file.split(".")[0] + "_processed.data"
                ff = open(file_to_create, "w")
                n = 1
                item = 0
                print("Creo il quaternione")
                # quats_dict = make_quaternion(mat)
                quats = make_quaternion(mat)
                print('Appendo la classe del dialetto al quaternione')
                dictionary ='\t' + dict_list[path_dict]
                # quats_dict.append(dictionary)
                quats_length = len(quats)

                while (item < quats_length):
                    if item == N_SPLIT * n:
                        ff.write(dictionary)
                        ff.write("\n")
                        n = n + 1
                    quat = quats[item]
                    for i in range(len(quat)):
                        if i == 3:
                            ff.write(str(quat[i]) + " ")
                        else:
                            ff.write(str(quat[i]) + ",")
                    item = item + 1
                x = int((n*N_SPLIT) - quats_length)
                for i in range(x):
                    if i < x-1:
                        ff.write("0,0,0,0 ")
                    else:
                        ff.write("0,0,0,0")

Analizzo la cartella DR3
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto al quaternione
Creo il quaternione
Appendo la classe del dialetto 

In [10]:
!zip -r quaternion_400.zip /content/drive/My\ Drive/COLAB\ Neural/Quaternion_400_Lamush

  

  adding: content/drive/My Drive/COLAB Neural/Quaternion_400_Lamush/ (stored 0%)
  adding: content/drive/My Drive/COLAB Neural/Quaternion_400_Lamush/SA2_177_processed.data (deflated 54%)
  adding: content/drive/My Drive/COLAB Neural/Quaternion_400_Lamush/SI1916_202_processed.data (deflated 54%)
  adding: content/drive/My Drive/COLAB Neural/Quaternion_400_Lamush/SA2_188_processed.data (deflated 54%)
  adding: content/drive/My Drive/COLAB Neural/Quaternion_400_Lamush/SX231_201_processed.data (deflated 54%)
  adding: content/drive/My Drive/COLAB Neural/Quaternion_400_Lamush/SA1_209_processed.data (deflated 54%)
  adding: content/drive/My Drive/COLAB Neural/Quaternion_400_Lamush/SI1371_183_processed.data (deflated 54%)
  adding: content/drive/My Drive/COLAB Neural/Quaternion_400_Lamush/SI1208_176_processed.data (deflated 54%)
  adding: content/drive/My Drive/COLAB Neural/Quaternion_400_Lamush/SI1173_210_processed.data (deflated 54%)
  adding: content/drive/My Drive/COLAB Neural/Quaternion_

In [0]:
!cp quaternion_400.zip /content/drive/My\ Drive/COLAB\ Neural/Quaternion_400_Lamush
